# Computing integrals


In [4]:
import numpy as np
import plotly.offline as py
from plotly.graph_objs import Scatter, Bar, Layout, Figure

py.init_notebook_mode(connected=True)

In [24]:
def f(x):
    return 1 - np.exp(-x)


def estimate_integral(samples, area):
    count = 0
    x, y = samples
    for i in range(len(x)):
        count += (y[i] < f(x[i]))

    return area * count / len(x)


def sample_rect(size=1):
    """Samples from a rectangle (0, 0), (1, f(1))."""
    x = np.random.random(size)
    y = np.random.random(size) * f(1)

    return x, y


def print_integral_summary(generate_sample, area, Ns):
    estimates = []
    stddevs = []
    for N in Ns:
        values = np.array([estimate_integral(generate_sample(N), area) for _ in range(1000)])
        value = np.mean(values)
        stddev = np.sqrt(np.var(values))
        error = np.sqrt(np.mean(np.power(values - ANALYTICAL_RESULT, 2)))

        print("N = {:5}\t I = {:f}\t stddev = {:f}\t error = {:f}".format(N, value,
                                                                stddev, error))
        estimates.append(value)
        stddevs.append(stddev)
    
    data = [Bar(
        x=["N = {}".format(n) for n in Ns],
        y=estimates,
        error_y=dict(type="data", array=stddevs)
    )]
    
    py.iplot(data)


## Estimate using rectangle

Here we compute an estimate of the integral using the accept-reject method. 

N is the number of sample points used,
I is the estimate of the integral, 
stddev the standard deviation of I done using 1000 attempts for ench value of N, 
error is the square root of mean square deviation of I from the (known) true value of the integral.

The same letters with the same meaning will be also used in the following parts.

In [25]:
ANALYTICAL_RESULT = np.exp(-1)

Ns = [10, 100, 1000]

print("Sampling from rectangle (0, 0), (1, f(1)).\n"
      "==========================================")
print_integral_summary(sample_rect, f(1), Ns)

Sampling from rectangle (0, 0), (1, f(1)).
N =    10	 I = 0.369727	 stddev = 0.099791	 error = 0.099808
N =   100	 I = 0.366649	 stddev = 0.030828	 error = 0.030853
N =  1000	 I = 0.367696	 stddev = 0.009930	 error = 0.009931


## Estimate under triangle


Explain why sampling x from p(x)=2x and then y unifomly in  [0, x] generates points uniformly distributed below the line x for x $\in [0,1]$

Let $F(x)$ be the integral of $f(x)=x$. 
A constant density of points in the region $(y<x, \ y>0,\ x\in[0,1])$ means first of all that for any fixed $x_{i}$ the number of points $N_{i}$ in the interval$ (x_{i}, x_{i}+\Delta)$ must be proportional to the area measured by $F(x_{i}+\Delta)-F(x_{i})$ then, inside each bin, the $N_{i}$ points must be uniformly distributed over the height of the bin i.e. between 0 and $f_{i}$. 

We will proove that the probability distribution of the sample along $x$ must be proportional to $f(x)$.
The uniform-density condition reads:

$$
\begin{equation}
\dfrac{F(x_{i}+\Delta)-F(x_{i})}{N_{i}}=const
\end{equation}
$$

If we set the value of the constant to be equal to $1/N_{tot}$, where $N_{tot}$ is the total number of points, we have in the $\lim_{\Delta\to 0}, \lim_{N\to\infty} $:

$$
\begin{equation}
\dfrac{F(x_{i}+\Delta)-F(x_{i})}{\Delta}\rightarrow f(x), \quad \dfrac{N_{i}}{N\Delta}\rightarrow p(x)
\end{equation}
$$

Why the error is lower on average?

Given a $f(x) \geq 0$ the accept-reject method actually computes an estimate of the mean $\theta$ of the bernuolli random variable that is set equal to 1 if the point fall below the graph of the function $f(x)$, and 0 otherwise. Knowing the total area $A$ in which the points are thrown the integral of the function is then $I=A\theta$. The estimate of $\theta$ is computed through the empirical average of the sample, which is an unbiased estimator with standard deviation $\sigma=(\theta(1-\theta)/N)^{1/2}$, we have:

$$
\begin{equation}
\hat{\theta}=\theta \pm \sqrt{\dfrac{\theta(1-\theta)}{N}}
\end{equation}
$$

If $\theta$ is of order 1, as reasonably should happen, then the average error at fixed $N$ is proportional to the standard deviation of the estimator and decreases as $\theta$ increases, or as the sample region approaches the function $f(x)$. This is what happens changing the rectangle into the triangle: the area of the triangular region is colser to the integral to be estimated, therefore the average error is lower than that of the rectangular sample area.
For completeness we can say that the average error is a decreasing function of $\theta$ in all the interval $[0,1]$ and is not proportional to $\sigma$.

What is usually referred as precision is the realtive error which is:

$$
\begin{equation}
e=\sqrt{\dfrac{1-\theta}{A^{2}\theta N}}
\end{equation}
$$

In [26]:
def sample_triangle(size=1):
    """Samples from a triangle y < x from (0, 0), to (1, 1)."""
    x = np.random.random(size)**0.5
    y = np.random.uniform(0, x, size)
    
    return x, y

print("Sampling from triangle (0, 0), (1, 1).\n"
      "======================================")
print_integral_summary(sample_triangle, 0.5, Ns)

Sampling from triangle (0, 0), (1, 1).
N =    10	 I = 0.366850	 stddev = 0.069703	 error = 0.069711
N =   100	 I = 0.367435	 stddev = 0.021428	 error = 0.021432
N =  1000	 I = 0.367908	 stddev = 0.007157	 error = 0.007157


### Sampling under g(x)

Set $g(x)=(1-e^{-1})x^{1/2}$, $f(x)=1-e^{-x}$ show that $f(x) \leq g(x) \quad \forall x\in[0,1]$.

Dim

$f,g \subset C^{\infty}(0,1)$ and $f(0)=g(0)=0$,  $f(1)=g(1)=1-e^{-1}$; if $f(x) \leq g(x) \quad \forall x\in[0,1]$, these points must be the only one in which the two functions are equal in $(0,1)$.
For the Cauchy's mean value theorem, for any ${a,b}$ for which$f(a)=g(a)$ and $f(b)=g(b)$ must exist at least a $c\in(a,b)$ that realizes $f'(c)=g'(c)$. We will now proove that in $(0,1)$ there is only one $c$ that realizes the equality of the first derivatives and therefore other points for which $f=g$ can't exist.

$f'(x)=g'(x)$ gives:

$$
\begin{equation}
e^{-x}=\dfrac{1-e^{-1}}{2x^{1/2}}\quad \Rightarrow \quad x^{1/2}e^{-x}=\dfrac{1-e^{-1}}{2}\doteq h(x)
\end{equation}
$$

Now the following inequalities:

$$
\begin{equation}
h(0)=0<(1-e^{-1})/2, \quad h(1)=e^{-1}>(1-e^{-1})/2
\end{equation}
$$

(the second being equivalent to $(3>e)$) imply, for the continuity of $h$, the existence of at least one point of intersection between $h$ and $(1-e^{-1})/2$.

We will show at the end of the proof that $h$ is concave in $(0,1)$. Given this let's choose the first $x^{*}$ for which $h(x)=(1-e^{-1})/2$. 

Then by definition of concavity we have:

$$
\begin{equation}
h(x^{*}+(1-x^{*})t)> h(x^{*})+(h(1)-h(x^{*}))t \quad \forall t \in [0,1]
\end{equation}
$$

but we have:

$$
\begin{equation}
h(x^{*})=\dfrac{1-e^{-1}}{2}, \quad h(1)-h(x^{*})>0
\end{equation}
$$

therefore

$$
\begin{equation}
h(x)>\dfrac{1-e^{-1}}{2} \forall x \in(x^{*},1]
\end{equation}
$$

and we conclude that there exist only one point $c\equiv x^{*}$ in $(0,1)$ for which $f'(c)=g'(c)$.

Finally $h$ is indeed concave since:

$$
\begin{equation}
h''(x)=\dfrac{e^{-x}}{x^{1/2}}\left(-1+x-\dfrac{1}{4x}\right)<0 \ \forall x\in[0,1]
\end{equation} 
$$




Show how to sample from the distribution $p_{X}(x)=3/2\sqrt{x}$.

Since the pdf is a power law it is esay to use the cumulative method to generate random numbers from it. The cumulative distribution function is:

$$
\begin{equation}
F(x)=x^{3/2} \quad x \in [0,1]
\end{equation}
$$

The prescription is then to take the inverse that will allow to pass from a unifom distribution to the $ p_{X}(x)$ we seek. More explicitly let $y_{i}$ be a random number from a uniform distribution in the interval $[0,1]$ then:

$$
\begin{equation}
x_{i}=F^{-1}(y_{i})=y_{i}^{2/3}
\end{equation}
$$

will be distributed according to $p_{X}(x)=3/2\sqrt{x}.$

In [31]:
def g(x):
    return (1 - np.exp(-1)) * x**0.5


def sample_g(size=1):
    """Samples uniformly between the x-axis and g(x)."""
    x = np.random.random(size)**(2/3)
    y = np.random.uniform(0, g(x), size)

    return x, y

print("Sampling from area under g(x).\n"
      "==============================")
print_integral_summary(sample_g, 2/3*(1 - np.exp(-1)), Ns)

Sampling from area under g(x).
N =    10	 I = 0.368568	 stddev = 0.043213	 error = 0.043218
N =   100	 I = 0.367313	 stddev = 0.014225	 error = 0.014236
N =  1000	 I = 0.368244	 stddev = 0.004445	 error = 0.004460


## Gaussian average

The integrand is an average weighted according to a Gaussian distribution. Since the joint pdf is the product of three standard gaussian distributions, to compute the integral we first generate $x_{i}, y_{i}, z_{i}$ each one from a standard gaussian distribution, then we compute the estimate with the following formula:

$$
\begin{equation}
I=\dfrac{\sum_{i}^{N }g(x_{i}, y_{i}, z_{i})}{N}, \quad g(x,y,z)=\vert \cos{\sqrt{x^{2}+y^{2}}} \vert \tanh{(x^{2}+y^{2}+z^{4})}
\end{equation} 
$$

In [19]:
def h(x, y, z):
    return abs(np.cos((x**2 + y**4)**0.5)) * np.tanh(x**2 + y**2 + z**4)


def gaussian_average(h, N):
    I = 0
    for i in range(N):
        x, y, z = np.random.normal(size=3)
        I += h(x, y, z) / N

    return I


for N in [10, 100, 1000, 10000, 100000, 1000000]:
    I = gaussian_average(h, N)
    print("The estimate of the integral with {} points is {}".format(N, I))

The estimate of the integral with 10 points is 0.5451194729460527
The estimate of the integral with 100 points is 0.4387700856719226
The estimate of the integral with 1000 points is 0.47671726182638613
The estimate of the integral with 10000 points is 0.47430935116048045
The estimate of the integral with 100000 points is 0.4770094126301008
The estimate of the integral with 1000000 points is 0.4752160200601276
